In [1]:
import json
import numpy as np
import pandas as pd
from itertools import product

In [2]:
DATA_SOURCE = "darknet"

In [3]:
pd.read_csv("data/2021/input/stacking_predictions/out/k3/test/idarkvec_20211221_fold00.csv").head()

,mirai,unk_bruteforcer,unk_spammer,shadowserver,driftnet,internetcensus,censys,rapid7,onyphe,netsystems,shodan,unk_exploiter,securitytrails,intrinsec,unknown,y_true
0,0.666667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.333333,mirai
1,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,mirai
2,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.000000,unk_spammer
3,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,mirai
4,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,censys


In [4]:
strats = [["features", "features"], ["nlp", "idarkvec"], ["gnn", "igcngru_features"]]

In [5]:
jp = "data/2022/input/skf/stratification/stratification.json"
with open(jp, 'r') as fd:
    splits = json.load(fd)

In [6]:
len(splits["20221021"][DATA_SOURCE]), len(splits["20221021"][DATA_SOURCE][0])

(10, 2)

In [7]:
days = list(splits.keys())
days.sort()
days

['20221021',
 '20221022',
 '20221023',
 '20221024',
 '20221025',
 '20221026',
 '20221027',
 '20221028',
 '20221029',
 '20221030',
 '20221031']

In [17]:
iters = product(days, strats, np.arange(10))
for day, (strat, label), fold in iters:
    file_path = f"data/2022/input/skf/predictions_stratified/{DATA_SOURCE}_{strat}_{day}_fold{fold}.csv"
    df = pd.read_csv(file_path)
    train_ips = splits[day][DATA_SOURCE][fold][0]
    test_ips = splits[day][DATA_SOURCE][fold][1]
    df_train = df[df.src_ip.isin(train_ips)]
    df_test = df[df.src_ip.isin(test_ips)]
    df_train.to_csv(
        f"data/2022/input/stacking_predictions/out/k3/{DATA_SOURCE}/train/{label}_{day}_fold0{fold}.csv"
    )
    df_test.to_csv(
        f"data/2022/input/stacking_predictions/out/k3/{DATA_SOURCE}/test/{label}_{day}_fold0{fold}.csv"
    )

In [18]:
pd.read_csv("data/2022/input/skf/predictions_stratified/honeypot_features_20221021_fold7.csv")

,src_ip,censys,driftnet,internetcensus,intrinsec,ipip,mirai,onyphe,rapid7,securitytrails,shadowserver,shodan,u_mich,unk_bruteforcer,unk_exploiter,unk_spammer,unknown,y_true
0,124.130.69.153,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,mirai
1,180.115.172.54,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,mirai
2,116.248.33.51,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,mirai
3,85.105.218.240,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,mirai
4,172.104.210.91,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18954,151.252.187.254,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,mirai
18955,143.110.177.98,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,unk_bruteforcer
18956,117.205.90.200,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,unk_bruteforcer
18957,74.204.129.194,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,unk_bruteforcer
